In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Dell\\OneDrive\\Desktop\\Real_projects\\End_to_end_ml_project_mlops\\research'

In [3]:
%pwd

'c:\\Users\\Dell\\OneDrive\\Desktop\\Real_projects\\End_to_end_ml_project_mlops\\research'

In [4]:
import os

In [5]:
os.chdir('../')

In [6]:
%pwd

'c:\\Users\\Dell\\OneDrive\\Desktop\\Real_projects\\End_to_end_ml_project_mlops'

In [7]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/codedestructed007/Ml_end_to_end_project.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'codedestructed007'
os.environ['MLFLOW_TRACKING_PASSWORD'] = "5780be4d27268796fc9f39b1b123b32d86ba1bac"


In [8]:
from dataclasses import dataclass
from pathlib import Path

In [9]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [10]:
from src.mlproject.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH,SCHEMA_FILE_PATH
from src.mlproject.utils.common import read_yaml,create_directories

In [11]:
class ConfigurationManager:
    def __init__(self,
                 config_filapath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filapath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            all_params= params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/codedestructed007/Ml_end_to_end_project.mlflow"
        )
        return model_evaluation_config

In [12]:
import json
import pandas as pd
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [13]:
class ModelEvaluation:
    def __init__(self,config : ModelEvaluationConfig):
        self.config = config
            
    def eval_metrics(self, actual,pred):
        rmse = r2_score(actual,pred)
        mse = mean_squared_error(actual,pred)
        mae = mean_absolute_error(actual,pred)
        return rmse,mae,mse
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        x_test = test_data.drop(self.config.target_column,axis=1)
        y_test = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted_qualities = model.predict(x_test)
            (rmse,mae,mse) = self.eval_metrics(y_test,predicted_qualities)
            
            # saving metrics as local
            scores = {'rmse': rmse , 'mae' : mae, 'r2' : rmse}
            with open(self.config.metric_file_name, 'w') as json_file:
                
                json.dump(scores,json_file)
                
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric('rmse',rmse)
            mlflow.log_metric('mae', mae)
            mlflow.log_metric('r2', rmse)
            
            if tracking_url_type_store != 'file':
                
                mlflow.sklearn.log_model(model,'model',registered_model_name='ElasticNet')
            else:
                mlflow.sklearn.log_model(model ,'model')
                
            
            
    

In [14]:
try: 
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

2023-09-18 17:15:41,500 - common - INFO - yaml file : {} loaded successfully
2023-09-18 17:15:41,505 - common - INFO - yaml file : {} loaded successfully
2023-09-18 17:15:41,509 - common - INFO - yaml file : {} loaded successfully
2023-09-18 17:15:41,509 - common - INFO - Created directory : artifacts
2023-09-18 17:15:41,514 - common - INFO - Created directory : artifacts/model_evaluation


c:\Users\Dell\OneDrive\Desktop\Real_projects\End_to_end_ml_project_mlops\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'ElasticNet' already exists. Creating a new version of this model...
2023/09/18 17:16:05 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticNet, version 2
Created version '2' of model 'ElasticNet'.
